In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import tensorflow as tf
%matplotlib inline
import nltk
from nltk.stem import PorterStemmer,WordNetLemmatizer
# import seaborn as sns
import time
import re
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
import os
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [58]:
def getTextFromFiles(df, data_path, depression, limit):
    """Return Data Frame """

    for file in os.listdir(data_path)[:limit]:
        with open(data_path + "/" + file, 'r', encoding="ISO-8859-1") as file1:
            file1 = file1.read()
            df = df.append({'text': file1, 'depression': int(depression)}, ignore_index=True)

    return df

In [59]:
data_path_d = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_depression"
data_path_nd = "/content/drive/My Drive/NLP Team/code/kerasData/reddit_non_depression"
df = pd.DataFrame(columns=['text', 'depression'])
df = getTextFromFiles(df, data_path_d, 1, 500)
df = getTextFromFiles(df, data_path_nd, 0, 500)


In [60]:
X = df['text'].to_numpy()
Y = df['depression'].to_numpy()
# le = LabelEncoder()
# Y = le.fit_transform(Y)
# Y = Y.reshape(-1,1)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

# max_words = 100
# max_len = 150

max_words = 1000
max_len = 150

tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)


In [61]:
print(X_train.shape)
print(np.array(sequences).shape)
# print(sequences)
print(sequences_matrix.shape)
print(Y_train.shape)
print(tok.word_index)

(850,)
(850,)
(850, 150)
(850,)
{'i': 1, 'to': 2, 'and': 3, 'the': 4, 'a': 5, 'my': 6, 'me': 7, 'of': 8, 'that': 9, 'she': 10, 'it': 11, 'in': 12, 'her': 13, 'but': 14, 'is': 15, 'with': 16, 'for': 17, 'was': 18, 'this': 19, 'have': 20, 'just': 21, 'so': 22, 'like': 23, 'he': 24, 'not': 25, 'on': 26, 'you': 27, "i'm": 28, 'about': 29, 'be': 30, 'do': 31, 'we': 32, 'at': 33, 'or': 34, 'out': 35, 'all': 36, 'as': 37, 'feel': 38, 'if': 39, 'when': 40, 'because': 41, 'know': 42, 'they': 43, 'friends': 44, "don't": 45, 'what': 46, 'had': 47, 'up': 48, 'how': 49, 'time': 50, 'him': 51, 'want': 52, 'been': 53, 'get': 54, 'friend': 55, 'even': 56, 'really': 57, 'one': 58, 'from': 59, 'am': 60, 'are': 61, 'would': 62, 'them': 63, 'life': 64, 'people': 65, 'has': 66, 'now': 67, 'no': 68, 'can': 69, 'an': 70, 'other': 71, 'never': 72, 'who': 73, 'then': 74, 'go': 75, 'myself': 76, "it's": 77, 'family': 78, 'things': 79, 'iâ\x80\x99m': 80, 'years': 81, 'his': 82, "i've": 83, 'think': 84, 'only': 8

In [63]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_words,50,input_length=max_len))
model.add(tf.keras.layers.LSTM(64))
model.add(tf.keras.layers.Dense(256,name='FC1'))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1,name='out_layer'))
model.add(tf.keras.layers.Activation('sigmoid'))

model.build(input_shape=sequences_matrix.shape)
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [64]:
# model = tf.keras.models.Sequential()
# model = RNN()
model.summary()
# model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation_10 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257       
_________________________________________________________________
activation_11 (Activation)   (None, 1)                

In [65]:
model.fit(sequences_matrix.astype(float), Y_train.astype(float), batch_size=4,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/10
170/170 [==============================] - 9s 53ms/step - loss: 0.6434 - accuracy: 0.6118 - val_loss: 0.5183 - val_accuracy: 0.7353
Epoch 2/10
170/170 [==============================] - 8s 50ms/step - loss: 0.3547 - accuracy: 0.8515 - val_loss: 0.3476 - val_accuracy: 0.8529
Epoch 3/10
170/170 [==============================] - 9s 51ms/step - loss: 0.3014 - accuracy: 0.9191 - val_loss: 0.5333 - val_accuracy: 0.8471


In [43]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [44]:
accr = model.evaluate(test_sequences_matrix.astype(float),Y_test.astype(float))

5/5 [==============================] - 0s 18ms/step - loss: 0.8771 - accuracy: 0.8133


In [45]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.877
  Accuracy: 0.813
